# Introduction
In `3. Part of Speech Tagging - Second attempt with RNN` I did Part of Speech (POS) tagging with several RNN's. My most successful was inspired by [Wang et al., 2015](https://arxiv.org/pdf/1510.06168.pdf), who used a model originally from [Graves, 2002](https://www.cs.toronto.edu/~graves/preprint.pdf):
Two LSTM layers with 93 hidden units, one doing a forward pass of the input and the other a backward pass. 

I encoded all words using word embeddings trained by [Glove](https://nlp.stanford.edu/projects/glove/). Glove is Stanfords embedding model.

In this notebook I will use a similar approach, but this time I will implement my model in PyTorch. Hopefully increased control will help me to learn an experiment.

# The data
I will be using the same training data for my tagger as in all my previous notebooks:
[Universal Dependencies - English Web Treebank](http://universaldependencies.org/treebanks/en_ewt/index.html), a CoNLL-U formart corpus with 254 830 words and 16 622 sentences in english *taken from various web media including weblogs, newsgroups, emails, reviews, and Yahoo! answers*.

## Load the Data
First lets load the training data and convert it to a python dictionary.
I use the [conllu](https://github.com/EmilStenstrom/conllu) python package to parse the CoNLL-U files to dictionaries.

In [18]:
import numpy as np
import conllu
import torch

Read the data.

In [19]:
directory = 'UD/UD_English-EWT'
with open('{}/en_ewt-ud-train.conllu'.format(directory), 'r', encoding='utf-8') as f:
    train_text = f.read()
    
directory = 'UD/UD_English-EWT'
with open('{}/en_ewt-ud-dev.conllu'.format(directory), 'r', encoding='utf-8') as f:
    dev_text = f.read()
    
directory = 'UD/UD_English-EWT'
with open('{}/en_ewt-ud-test.conllu'.format(directory), 'r', encoding='utf-8') as f:
    test_text = f.read()

Convert it to a dictionary.

In [3]:
train_dict = conllu.parse(train_text)
dev_dict = conllu.parse(dev_text)
test_dict = conllu.parse(test_text)

Count sentences and tokens.

In [4]:
from functools import reduce

n_train_sentences = len(train_dict)
n_train_tokens = reduce(lambda x, y: x + len(y), train_dict, 0)

print("The training set contains {} sentences and {} tokens".format(n_train_sentences, n_train_tokens))

The training set contains 12543 sentences and 204607 tokens


In [5]:
train_sentences = [[token['form'] for token in sentence] for sentence in train_dict]
train_labels = [[token['upostag'] for token in sentence] for sentence in train_dict]

dev_sentences = [[token['form'] for token in sentence] for sentence in dev_dict]
dev_labels = [[token['upostag'] for token in sentence] for sentence in dev_dict]

test_sentences = [[token['form'] for token in sentence] for sentence in test_dict]
test_labels = [[token['upostag'] for token in sentence] for sentence in test_dict]

In [6]:
pos_tags = list(set(reduce(lambda x, y: x + y, train_labels)))

In [7]:
n_labels = len(pos_tags)
n_labels

17

# Feature Engineering
[Wang et al.](https://arxiv.org/pdf/1510.06168.pdf) showed that a bidirectional LSTM network could achieve state of the art performance without using any morphological features, they only used these features:
* Word embedding of the word (cast to lower case). Embeddings trained by the same architecture, but on another task.
* Suffix of length two, one-hot encoded
* Wether the word is all caps, lower case, or has an initial capital letter. One-hot encoded.

Other papers, like [Xiao et al.](https://arxiv.org/abs/1809.01997), complement word embeddings with character embeddings. 
It would be interesting to experiment with this.

## Glove
I will opt to use the 100 dimensional Glove 6B data. My vocabulary will be exactly the words inside the pretrained model.

In [8]:
glove_path = 'glove/glove.6B/glove.6B.100d.txt'
embeddings = {}
token_index = {}
index_token = {}
with open(glove_path, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        tok, *vec = line.split()
        embeddings[tok] = np.array(vec, dtype='float32')
        # Reserve index 0 for padding
        token_index[tok] = i + 1
        index_token[i+1] = tok

I know from my last notebook that my data has many words that are OOV for Glove. I adressed this by pre-processing my data. In this notebook I will condense my preprocessing to a single method, check the other one for more rationale on why I apply each step.

I will apply pre-processing to all three data sets: train, dev and test. As the preprocessing is not trained on the data this could be done on new data in production as well.

In [9]:
import re
def allign_to_vocab(vocab, sentences):
    
    # Find all OOV tokens
    oov = find_oov(vocab, sentences)
    print("%d OOV tokens before processing" % len(oov))
    
    # Convert to lower case
    sentences = [[token.lower() for token in sentence] for sentence in sentences]
    
    oov = find_oov(vocab, sentences)
    print("%d OOV tokens after converting to lower case" % len(oov))
    
    # Replace URL's with 'url'
    sentences = [[convert_url(token) for token in sentence] for sentence in sentences]
    
    oov = find_oov(vocab, sentences)
    print("%d OOV tokens after converting urls" % len(oov))
    
    # Build spelling correction dictionary
    # Search for word in vocabulary words within 1 Levensthein Damerau distance
    new_spelling = dict([(word, find_one_neighbour(word, embeddings)) for word in oov])
    sentences = [[new_spelling[token] if token in new_spelling else token for token in sentence] for sentence in sentences]
    
    oov = find_oov(vocab, sentences)
    print("%d OOV tokens after spelling correction" % len(oov))
    
    # Replace OOV words with 'unk'
    # See https://stackoverflow.com/questions/49239941/what-is-unk-in-glove-6b-50d-txt
    sentences = [['unk' if token in oov else token for token in sentence] for sentence in sentences]
    
    return sentences

def find_oov(vocab, sentences):
    oov = []
    for sentence in sentences:
        for tok in sentence:
            if tok not in vocab:
                oov.append(tok)
    return oov
     
def convert_url(token):
    # Match words starting with www., http:// or https://
    if re.match(r'^(?:https{0,1}\:\/\/.*|www\.*)', token):
        return "url"
    else:
        return token

# Checks for vocabulary words within 1 Damerau Levenstein distance and returns the first match
# Logic inspired by http://norvig.com/spell-correct.html
def find_one_neighbour(word, vocab):
    
    ascii_vocab = [str(chr(i)) for i in range(32, 127)]
    
    # Tuples with all possible splits of word
    splits = [(word[:i], word[i:]) for i in range(len(word))]
    
    # All words generated by deleting one character
    for L, R in splits:
        candidate = L + R[1:] if R else None
        if candidate in vocab:
            return candidate
    # All words generated by swapping two characters in word
    for L, R in splits:
        candidate = L + R[1:] if R else None
        if candidate in vocab:
            return candidate
    # All words generated by swapping two characters in word
    for L, R in splits:    
        candidate = L + R[1] + R[0] + R[2:] if len(R) > 1 else None
        if candidate in vocab:
            return candidate
    # All words generated by inserting a character in word
    for L, R in splits:
        for c in ascii_vocab:    
            candidate = L + c + R 
            if candidate in vocab:
                return candidate
    # All words generated by replacing a character in word
    for L, R, in splits:
        for c in ascii_vocab:    
            candidate = L + c +R[1:] if R else None
            if candidate in vocab:
                return candidate
        
    return word
    

In [10]:
%%time
print("Training set:")
train_sentences = allign_to_vocab(embeddings, train_sentences)

print("\nDev set:")
dev_sentences = allign_to_vocab(embeddings, dev_sentences)

print("\nTest set:")
test_sentences = allign_to_vocab(embeddings, test_sentences)

Training set:
31192 OOV tokens before processing
2442 OOV tokens after converting to lower case
2310 OOV tokens after converting urls
1066 OOV tokens after spelling correction

Dev set:
4362 OOV tokens before processing
463 OOV tokens after converting to lower case
424 OOV tokens after converting urls
215 OOV tokens after spelling correction

Test set:
4589 OOV tokens before processing
522 OOV tokens after converting to lower case
483 OOV tokens after converting urls
267 OOV tokens after spelling correction
Wall time: 12.5 s


## Encoding the data
I will encode the targets and the tokens as integers.

### Labels
First build a map from pos tag to an index. Reserve 0 for Padding.

In [20]:
pos_tag_index = {}
for i, pos in enumerate(pos_tags):
    pos_tag_index[pos] = i + 1

In [21]:
Y_train = [torch.from_numpy(np.asarray([pos_tag_index[pos] for pos in sentence])).long()  for sentence in train_labels]
Y_dev = [torch.from_numpy(np.asarray([pos_tag_index[pos] for pos in sentence])).long()  for sentence in dev_labels]
Y_test = [torch.from_numpy(np.asarray([pos_tag_index[pos] for pos in sentence])).long()  for sentence in test_labels]

### Tokens
I already built the mapping from token to index. I also replaced all OOV tokens with `unk`, so all words will be in the vocabulary. 

In [44]:
X_train = [torch.from_numpy(np.asarray([token_index[tok] for tok in sentence])).long() for sentence in train_sentences]
X_dev = [torch.from_numpy(np.asarray([token_index[tok]  for tok in sentence])).long()  for sentence in dev_sentences]
X_test = [torch.from_numpy(np.asarray([token_index[tok] for tok in sentence])).long()  for sentence in test_sentences]

# RNN

## Embedding Layer
Create a frozen embedding layer from the Glove data.

In [23]:
import torch

Load the data into a tensor.

In [24]:
embedding_dims = 100
embedding_matrix = torch.from_numpy(np.array(list(embeddings.values())))
n_words = len(embedding_matrix)

Create a frozen embedding layer.

In [25]:
import torch.nn as nn

In [26]:
def create_embeddings(weights, frozen=True):
    
    embedding_layer = nn.Embedding(*weights.shape, _weight=embedding_matrix)
    embedding_layer.weight.requires_grad = frozen
    
    return embedding_layer

In [27]:
embedding_layer = create_embeddings(embedding_matrix)

## BLSTM 1
Let's implement the BLSTM introduced by Graves, with our Glove embeddings as the first layer.

In [28]:
import torch.nn.functional as F

In [287]:
class BLSTM1(nn.Module):
    def __init__(self, lstm_dim, n_classes, embedding_weights):
        super(BLSTM1, self).__init__()
        
        # Variables
        self.lstm_dim = lstm_dim
        self.vocab_size, self.embedding_dim = embedding_weights.shape
        self.n_classes = n_classes
        
        # Layers
        self.embedding = create_embeddings(embedding_weights)
        self.lstm = nn.LSTM(self.embedding_dim, lstm_dim, batch_first=True, bidirectional=True)
        self.output = nn.Linear(self.lstm_dim * 2, self.n_classes + 1)
        
    def forward(self, padded_batch, sequence_lengths):
        
        # Embeddings
        embedded = self.embedding(padded_batch)#.unsqueeze(0)
        
         # Pack Padded Batch
        total_length = padded_batch.size(1)
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, batch_first=True, lengths=sequence_lengths)
        
        # LSTM
        out, _ = self.lstm(packed_embedded)
        
        # Reverse Packing
        out, _ = nn.utils.rnn.pad_packed_sequence(out, batch_first=True, total_length=total_length)
        
        # Fully Connected
        out = F.log_softmax(self.output(out), 2)
        return out
        

In [351]:
blstm1 = BLSTM1(96, n_labels, embedding_matrix)

In [352]:
import torch.optim as optim

In [353]:
criterion = nn.NLLLoss(ignore_index=0)
optimizer = optim.Adam(blstm1.parameters())

In [354]:
class PaddedGenerator():
    def __init__(self, x_set, y_set, batch_size):
        
        self.batch_size = batch_size
        self.x, self.y, self.sequence_lengths = [], [], []
        # Pad Data so all batches have sequences of equal length
        for idx in range(int(np.ceil(len(x_set) / float(self.batch_size)))):
            batch_x = x_set[idx * self.batch_size:(idx + 1) * self.batch_size]
            batch_y = y_set[idx * self.batch_size:(idx + 1) * self.batch_size]
            
            # The input is sorted so the first sequence is the longest
            max_len = len(batch_x[0])
            batch_sequence_lengths = torch.Tensor(list(map(len, batch_x))).int()
            batch_x = nn.utils.rnn.pad_sequence(batch_x, batch_first=True)
            batch_y = nn.utils.rnn.pad_sequence(batch_y, batch_first=True)
            
            self.x.append(batch_x)
            self.y.append(batch_y)
            self.sequence_lengths.append(batch_sequence_lengths)
            
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx], self.sequence_lengths[idx]

In [361]:
from tqdm import tqdm

# Function inspired by https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
def train_model(model, train_generator, optimizer, criterion, dev_generator=None, epochs = 2):
    
    # Only enter the validation state if there is a validation_loader
    phases = ['train']
    data_dict = {'train' : train_generator} 
    if dev_generator:
        phases.append('val')
        data_dict['val'] = dev_generator
        
    for epoch in range(epochs):
        
        print('Epoch {}/{}'.format(epoch + 1, epochs))
        print('-' * 10)

        for phase in phases:
            
            data = data_dict[phase]
            
            # Only update model weights based on the training data
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            words_processed = 0
            
            for seq, labels, sequence_lengths in tqdm(data, total = len(data)):
                
                
                #labels = torch.autograd.Variable(labels).type(torch.LongTensor)

                optimizer.zero_grad()
                
                # Only track history during training
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(seq, sequence_lengths)
                    loss = criterion(outputs.view(-1, n_labels + 1), labels.view(-1))
                    predictions = torch.argmax(outputs, dim=2)
                    
                    # Only perform backpropagation during training
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                # Save statistics
                running_loss += loss.item() * torch.sum(sequence_lengths).item()
                # Only count correct classifications when the label is not padding
                running_corrects += torch.sum((predictions == labels.data) * (labels.data != 0))
                words_processed += torch.sum(sequence_lengths)
                
                #return predictions, labels, words_processed
                
            epoch_loss = running_loss / words_processed.item()
            epoch_acc = running_corrects.item() / words_processed.item()
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
                
            

In [356]:
X_train = sorted(X_train, reverse=True, key=len)
Y_train = sorted(Y_train, reverse=True, key=len)

In [357]:
X_dev = sorted(X_dev, reverse=True, key=len)
Y_dev = sorted(Y_dev, reverse=True, key=len)

In [358]:
X_test = sorted(X_test, reverse=True, key=len)
Y_test = sorted(Y_test, reverse=True, key=len)

In [360]:
train_padded = PaddedGenerator(X_train, Y_train, batch_size=126)
dev_padded = PaddedGenerator(X_dev, Y_dev, batch_size=126)

In [362]:
train_model(blstm1, train_padded, optimizer, criterion, dev_padded)

Epoch 1/2
----------


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:22<00:00,  1.22it/s]


train Loss: 1.9682 Acc: 0.4418


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:01<00:00, 13.81it/s]


val Loss: 2.2596 Acc: 0.3310
Epoch 2/2
----------


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:21<00:00,  1.23it/s]


train Loss: 0.7649 Acc: 0.7754


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:01<00:00, 14.22it/s]


val Loss: 0.5118 Acc: 0.8401


In [363]:
train_model(blstm1, train_padded, optimizer, criterion, dev_padded, epochs=5)

Epoch 1/5
----------


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:25<00:00,  1.17it/s]


train Loss: 0.3798 Acc: 0.8869


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:01<00:00, 13.38it/s]


val Loss: 0.4255 Acc: 0.8665
Epoch 2/5
----------


...

val Loss: 0.3761 Acc: 0.8892
Epoch 5/5
----------


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:09<00:00,  1.43it/s]


train Loss: 0.1728 Acc: 0.9489


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:01<00:00, 15.76it/s]


val Loss: 0.3810 Acc: 0.8915


In [364]:
train_model(blstm1, train_padded, optimizer, criterion, dev_padded, epochs=5)

Epoch 1/5
----------


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:15<00:00,  1.33it/s]


train Loss: 0.1524 Acc: 0.9552


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 16.62it/s]


val Loss: 0.3856 Acc: 0.8940
Epoch 2/5
----------


...

val Loss: 0.3951 Acc: 0.8990
Epoch 5/5
----------


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:24<00:00,  1.18it/s]


train Loss: 0.0998 Acc: 0.9711


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:01<00:00, 13.27it/s]


val Loss: 0.3992 Acc: 0.8999


In [366]:
torch.save(blstm1.state_dict(), 'pytorch_models/blstm1.pt')

Similar results as my last BLSTM.

It would be interesting to try using a GRU instead of LSTM, apply some regularization like Dropout, and perhaps experiment with character level features.